<a href="https://colab.research.google.com/github/kalyan6305/FMML-22B21A4531/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [2]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

In [13]:
# Q1. Increasing the percentage of Validation Set
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 90/100) # Increased from 75 to 90
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation (75%) accuracy of nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation (75%) accuracy using random classifier is ", valAccuracy)

print()
# Decreasing the percentage of Validation Set

traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 10/100) # Decreased from 75 to 10
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation (10%) accuracy of nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation (10%) accuracy using random classifier is ", valAccuracy)


Validation (75%) accuracy of nearest neighbour is  0.3504424778761062
Validation (75%) accuracy using random classifier is  0.16283185840707964

Validation (10%) accuracy of nearest neighbour is  0.29866630742287487
Validation (10%) accuracy using random classifier is  0.1669136467735417


In [14]:
# Q2. Let us take 50% of the dataset as the Train set
# 25% of the dataset as the validation set
# 25% of the dataset as the test set
traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 50/100)
valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)

print(f"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}")

testpred = NN(testdata, testlabel, valdata)
testpred2 = RandomClassifier(testdata, testlabel, valdata)

accu = Accuracy(vallabel, testpred)
accu2 = Accuracy(vallabel, testpred2)
print(f"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}")
print(f"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}")


Percent of Train Data in the dataset: 50.24
Percent of Test Data in the dataset: 24.49
Percent of Validation Data in the dataset: 25.27
Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.3079578139980825
Accuracy (Random Classifier) of Test Set using Validation Set: 0.1695110258868648


In [15]:
# Small Training Set, Large Validation Set
traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 10/100)
valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)

print(f"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}")

testpred = NN(testdata, testlabel, valdata)
testpred2 = RandomClassifier(testdata, testlabel, valdata)

accu = Accuracy(vallabel, testpred)
accu2 = Accuracy(vallabel, testpred2)
print(f"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}")
print(f"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}")


Percent of Train Data in the dataset: 10.0
Percent of Test Data in the dataset: 45.29
Percent of Validation Data in the dataset: 44.71
Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.3347420892934547
Accuracy (Random Classifier) of Test Set using Validation Set: 0.1681837884698743


In [16]:
# Large Training Set, Small Validation Set
traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 80/100)
valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)

print(f"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}")

testpred = NN(testdata, testlabel, valdata)
testpred2 = RandomClassifier(testdata, testlabel, valdata)

accu = Accuracy(vallabel, testpred)
accu2 = Accuracy(vallabel, testpred2)
print(f"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}")
print(f"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}")


Percent of Train Data in the dataset: 80.43
Percent of Test Data in the dataset: 10.01
Percent of Validation Data in the dataset: 9.56
Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.30410542321338063
Accuracy (Random Classifier) of Test Set using Validation Set: 0.16421692853522554


In [17]:
# Balanced Datasets
traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 50/100)
valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)

print(f"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}")
print(f"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}")

testpred = NN(testdata, testlabel, valdata)
testpred2 = RandomClassifier(testdata, testlabel, valdata)

accu = Accuracy(vallabel, testpred)
accu2 = Accuracy(vallabel, testpred2)
print(f"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}")
print(f"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}")


Percent of Train Data in the dataset: 49.54
Percent of Test Data in the dataset: 25.22
Percent of Validation Data in the dataset: 25.24
Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.3211132437619962
Accuracy (Random Classifier) of Test Set using Validation Set: 0.16065259117082534


In Train-Validation-Set Split we get highest accuracy for the 10-45-45 Split, But in Reality the performance of the model is decreased becuase the train set is small when compared to the test and validation set.

50-25-25 Split is the best in my opinion

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [18]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [19]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.3387104262363516
test accuracy is  0.8680115273775216


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits in a cross-validation process does give more consistent and reliable results. Because it reduces Variance of the data.

(With a single validation split, the model's performance may be highly influenced by the specific data points included in that split, leading to variance in the performance estimate. Averaging over multiple splits smooths out this variance.)

Does it give more accurate estimate of test accuracy?

Cross-validation provides a good estimate of how well your model should generalize to new, unseen data, which is valuable for model selection, hyperparameter tuning, and assessing generalization performance.

In [20]:
# Q3. What is the effect of the number of iterations on the estimate?
# Do we get a better estimate with higher iterations?
# Let's test with the iterations of 100

print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 100, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata) # Not to mention, it took 4m to execute
print('test accuracy(100 iterations) is ',Accuracy(testlabel, testpred))


Average validation accuracy is  0.3400074582036098
test accuracy(100 iterations) is  0.8680115273775216


In [21]:
# testing with 5 iterations
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 5, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy(5 iterations) is ',Accuracy(testlabel, testpred))

# Iterations are used to increase the accuracy of the model, but to some extent.
# Increase in the iterations can lead to increase in runtime, accuracy is same in all the test cases.


Average validation accuracy is  0.3419157526642346
test accuracy(5 iterations) is  0.8680115273775216


In [22]:
# Q4.Can we deal with a very small train dataset or validation dataset by increasing the iterations?
# Here I take 10% of the dataset as the train data
print("One Iteration\n")
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 0.01, 1, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy(1 iterations) is ',Accuracy(testlabel, testpred))
print("50 Iterations\n")
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 0.01, 1, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy(50 iterations) is ',Accuracy(testlabel, testpred))
print("100 Iterations\n")
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 0.01, 1, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy(100 iterations) is ',Accuracy(testlabel, testpred))


One Iteration

Average validation accuracy is  0.2835474006116208
test accuracy(1 iterations) is  0.8680115273775216
50 Iterations

Average validation accuracy is  0.2769645372695535
test accuracy(50 iterations) is  0.8680115273775216
100 Iterations

Average validation accuracy is  0.25217338067833966
test accuracy(100 iterations) is  0.8680115273775216


**Increasing the number of iterations (folds) in cross-validation can help when dealing with a very small training dataset or validation dataset to some extent, but it may not completely overcome the limitations of a small dataset**

Sometimes, the accuracy fluctuates, this is because Insufficient training data can lead to models that underfit and do not learn the underlying patterns effectively.

As we increase the iterations, we increases the runtime

Small datasets often suffer from sampling bias, where the limited data may not be representative of the overall population. More iterations may not fully address this issue

